In [3]:
import os
import gc
import re

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Layer, Input, Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 

In [4]:
# GradientReversalLayer for adversarial models
class GradientReversalLayer(Layer):
    def __init__(self, lambda_, **kwargs):
        super(GradientReversalLayer, self).__init__(**kwargs)
        self.lambda_ = lambda_

    @tf.custom_gradient
    def call(self, x):
        def grad(dy):
            return -self.lambda_ * dy
        return x, grad

    def get_config(self):
        config = super().get_config()
        config.update({
            "lambda_": self.lambda_
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



# Load the custom objects
custom_objects = {"GradientReversalLayer": GradientReversalLayer}

# Load the model
model_path = 'saved_data/models/adversarial-training_medium-masked_mobilenetv3small-finetuned_v3/adversarial-training_mobilenetv3small_fine-tuning.keras'
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)